# No Framework Notebook

In this notebook, we will explore yet another approach to process a user question, this time no framework would be used (SK). 
There would be two calls for LLM to classify and understand the user question, the second call (could be optional) would be to validate the generated answer does indeed answer the user question.

As before we have couple of methods which are used to obtain the actual answers (not from the LLM) - in our case both methods are simulating a response.

This notebook uses basic capabilities to interface with azure open ai. It uses `key` authentication to access the service.
It uses the Azure OpenAI SDK 2.1.0.

In [ ]:
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget: DotNetEnv, 2.5.0"

using Azure;

using DotNetEnv;

using System.IO;
using System.Text.Json;
using System.ClientModel;

using Azure.AI.OpenAI;
using Azure.AI.OpenAI.Chat;

using OpenAI.Chat;

using OpenAI.Embeddings;

## Reading Configuration

In [ ]:
string configurationFile = "../config/config.env";

Env.Load(configurationFile);

string apiKey = Env.GetString("SK_OPENAI_APIKEY");
string endpoint = Env.GetString("SK_OPENAI_ENDPOINT");
string chatCompletionDeployment = Env.GetString("SK_OPENAI_CHATCOMPLETION_DEPLOYMENT_DEFAULT");
string embeddingDeploymentName = Environment.GetEnvironmentVariable("EMBEDDING_DEPLOYMENTNAME") ?? "EMBEDDING_DEPLOYMENTNAME not found";

## Create Azure OpenAI Client

In [ ]:
AzureKeyCredential azureKeyCredential = new AzureKeyCredential(apiKey);
AzureOpenAIClient openAIClient = new AzureOpenAIClient(new Uri(endpoint), azureKeyCredential);
    
Console.WriteLine($"OpenAI Client created: {endpoint} with: {chatCompletionDeployment} and {embeddingDeploymentName} deployments");

## Open AI Helper Methods

### GetEmbeddingAsync

We have this method, although we are not using it in this notebook, it is used to get the embedding of the text.

In [ ]:
async Task<float[]> GetEmbeddingAsync(AzureOpenAIClient _openAIClient,string textToBeVecorized)
{
    // Prepare the embeddings options with the user story\n",
    EmbeddingClient embeddingClient = _openAIClient.GetEmbeddingClient(embeddingDeploymentName);
    ClientResult<OpenAIEmbedding> embeddingResult = await embeddingClient.GenerateEmbeddingAsync(textToBeVecorized);   
    float[] response = embeddingResult?.Value?.ToFloats().ToArray() ?? new float[0];
    return response;
}

Usage of this method is as follows:



In [ ]:
    float [] xx = await GetEmbeddingAsync(openAIClient,"sample text to embed");
    Console.WriteLine($"Embedding: {xx.Length}");

### CallOpenAIAsync

Performs the actual call to the `completion` endpoint of the Azure OpenAI service.

In [ ]:
async Task<string> CallOpenAIAsync(AzureOpenAIClient _openAIClient, string prompt, string systemMessage, bool jsonResponse = true)
{
    // Get the chat client (using your deployment or model name)
    ChatClient chatClient = _openAIClient.GetChatClient(chatCompletionDeployment);

    ChatCompletionOptions chatComletionOptions = new ChatCompletionOptions(){
        MaxOutputTokenCount = 150,
        Temperature = 0.7f,
        TopP = 1.0f,
        FrequencyPenalty = 0.7f,
        PresencePenalty = 0.7f,

    };

    chatComletionOptions.ResponseFormat = jsonResponse ? ChatResponseFormat.CreateJsonObjectFormat() : ChatResponseFormat.CreateTextFormat();

    // Prepare your messages
    ChatMessage[] messages = new ChatMessage[]
    {
        new SystemChatMessage(systemMessage),
        new UserChatMessage(prompt)
    };

    // Call the chat completions endpoint with parameters directly
    ChatCompletion completions = await chatClient.CompleteChatAsync(        
    messages: messages, 
    options: chatComletionOptions);

    // Get the text from the first completion choice
    // var resp = completions.Content[0];
    
    string result = completions.Content[0].Text;
    return result;
}

## Helper Methods & Classes

### ProcessLogger

Used to provide an execution log of the process.

In [ ]:
public class ProcessLogger
{
    private readonly List<string> _logs = new List<string>();

    // Log a generic message.
    public void Log(string message)
    {
        _logs.Add($"log: {message}");
    }

    // Optionally, log specific decision messages.
    public void LogDecision(string decision)
    {
        _logs.Add($"Decision: {decision}");
    }

    // Flush out all logs (for example, to the console) and then clear the log.
    public async Task FlushLogsAsync()
    {
        Console.WriteLine("=== Process Log Start ===");
        foreach (var log in _logs)
        {
            Console.WriteLine(log);
        }
        Console.WriteLine("=== Process Log End ===");
        _logs.Clear();
        await Task.CompletedTask;
    }
}

### Simulated Methods : GetSportEventScore, GetSportEventWinner

These methods are used to simulate a call to an external data source to obtain the requested information.

In [ ]:
public string GetSportEventScore(ProcessLogger logger,string sportEventName = "", string sportEventYear = "" )
{
    // Implement the logic to get the result of the sport event.
    logger.Log($"\tNot-Plugin: 'GetSportEventScore({sportEventName}, {sportEventYear})' called... will return '24:1'");
    return "24:1";
}

public string GetSportEventWinner(ProcessLogger logger,string sportEventName = "", string sportEventYear = "")
{  
    logger.Log($"\tNot-Plugin: 'GetSportEventWinner({sportEventName}, {sportEventYear})' called... will return 'Munich Flying Dolphins'");
    // Implement the logic to get the winner of the sport event.
    return "Munich Flying Dolphins";
}

## Prompts

Here we use 2 prompts 

- Validate the final response before sending it back (guardrail) 
- Understand the intent, and validate the question is within the domain it can answer

In [ ]:
string systemMessageValidateAnswer = 
@"You are an AI assistant that ensure answers match the question asked.
You only validate the domain, questions and answer may be in the future or factually incorrect, but still valid answer. 
The response JSON must have this structure:
{
  ""Answer"": ""Yes or No"",
  ""Explanation"": ""<A very short explanation>""
} 
";
string systemMessageUnderstandQuestion = 
@"You are an AI assistant that validate the question asked is about a sport event.
The response JSON must have this structure:
{
  ""Answer"": ""Yes or No"",
  ""SportEvent"": ""<Sport event name>"",
  ""Year"": ""<Year>"",
  ""Action"": ""<find_winner, find_score, find_winner_and_score>"",
  ""Explanation"": ""<A very short explanation>""
} 
";

## Helper classes to serialize LLM response

In [ ]:
using System.Text.Json.Serialization;

// Base class representing a generic prompt response.
public class PromptResponse
{
    [JsonPropertyName("Answer")]
    public string Answer { get; set; }
    
    [JsonPropertyName("Explanation")]
    public string Explanation { get; set; }
}

// Derived class for responses that include sports event details.
public class SportsPromptResponse : PromptResponse
{
    [JsonPropertyName("SportEvent")]
    public string SportEvent { get; set; }
    
    [JsonPropertyName("Year")]
    public string Year { get; set; }
    
    [JsonPropertyName("Action")]
    public string Action { get; set; }
}

## RunProcess

This is the entry point into the process flow. In this solution, we are using the `ProcessLogger` to log the process flow. Per user question the flow would run as follows:

- Understand the user question
- Validate the user question
- Generate the answer
- [Validate the answer] - optional


In [ ]:
async Task<string> RunProcess(AzureOpenAIClient _openAIClient, string userQuestion, ProcessLogger logger, bool validate = true)
{
    // step 1: validate the question, return an object that can be used programmatically
    var promptResponse = await CallOpenAIAsync(_openAIClient, userQuestion, systemMessageUnderstandQuestion, true);
    SportsPromptResponse response = JsonSerializer.Deserialize<SportsPromptResponse>(promptResponse);
    if (response == null || response.Answer != "Yes" || response.SportEvent == null || response.Year == null || response.Action == null)
    {
        logger.LogDecision($"Invalid question: {userQuestion}");
        return "Invalid question";
    }
    
    logger.Log($"Sport event: {response.SportEvent}, Year: {response.Year}, Action: {response.Action}");
    string sportEventWinner = string.Empty;
    string sportEventScore = string.Empty;
    string answer = string.Empty;

    logger.LogDecision($"Action: {response.Action}");
    // step 2: based on the actions, retrieve the information, draft the answer to the user question
    switch (response.Action)
    {
        case "find_winner":
            sportEventWinner = GetSportEventWinner(logger,response.SportEvent, response.Year);
            answer = $"The winner of the {response.SportEvent} in {response.Year} was {sportEventWinner}";
            break;
        case "find_score":
            sportEventScore = GetSportEventScore(logger,response.SportEvent, response.Year);
            answer = $"The score of the {response.SportEvent} in {response.Year} was {sportEventScore}";
            break;
        case "find_winner_and_score":
            sportEventWinner = GetSportEventWinner(logger,response.SportEvent, response.Year);
            sportEventScore = GetSportEventScore(logger, response.SportEvent, response.Year);
            answer = $"The winner of the {response.SportEvent} in {response.Year} was {sportEventWinner} with a score of {sportEventScore}";
            break;            
        default:
            logger.LogDecision($"Invalid action: {response.Action}");
            return "Invalid action";
    }   
    logger.Log($"Un validated answer: {answer}");
    // step 3: validate the answer before returning to the user
    if (validate)
    {
        var validateResponse = await CallOpenAIAsync(_openAIClient, answer, systemMessageValidateAnswer, true);
        // Console.WriteLine($"Validation response: {validateResponse}");
        PromptResponse validateRes = JsonSerializer.Deserialize<PromptResponse>(validateResponse);
        if (validateRes == null || validateRes.Answer != "Yes")
        {
            return "I could not draft an answer!";
        }
        logger.LogDecision($"Validated answer: {answer}");
    }

    return answer;

}

## Testing the process

Starting with some basic question:

In [ ]:
var userQuestion = "Who won the Super Sports Championship 2025, i cannot recall the score can u help?";
var logger = new ProcessLogger();
var result = await RunProcess(openAIClient, userQuestion, logger,true);
await logger.FlushLogsAsync();

Console.WriteLine($"User question: {userQuestion}");
Console.WriteLine($"Answer: {result}");

## Testing with more questions

The following is a method to run multiple user requests and measure the time taken to process each request.

In [ ]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.Linq;
using System.Threading;
using System.Threading.Tasks;

public async Task TestPerformanceConcurrent(AzureOpenAIClient openAIClient, bool validate = true)
{
    // Define a list of 20 questions.
    List<string> questions = new List<string>
    {
        "Who won the Super Sports Championship 2025?",
        "What was the score and who won the Euroleague of 2024?",
        "Who won the Super Sports Championship 2023?",
        "What was the final score of the Super Sports Championship 2022?",
        "Who won the Euroleague 2023?",
        "What was the decisive score of the Euroleague final in 2022?",
        "Who clinched the title in the Super Sports Championship 2024?",
        "Who was victorious in the Euroleague of 2025?",
        "What was the outcome of the Super Sports Championship 2021?",
        "Who won the Euroleague 2021?",
        "Who won the Super Sports Championship 2020?",
        "What was the score in the Euroleague of 2020?",
        "Who won the World Cup 2022?",
        "What was the final score of the International Grand Slam 2022?",
        "Who secured victory in the Continental Cup 2023?",
        "What was the final result of the National League 2024?",
        "Who took the championship in the Global Tournament 2025?",
        "What was the decisive score in the Global Tournament 2025?",
        "Who emerged as the winner in the Intercontinental Cup 2024?",
        "What was the score of the Euroleague final in 2024?"
    };

    // Create an instance of the logger.
    var logger = new ProcessLogger();
    logger.Log("Concurrent performance test started.");

    // Prepare a list to hold tasks, each returning the processing time in milliseconds.
    List<Task<long>> tasks = new List<Task<long>>();

    // Use a semaphore to limit the concurrency to 5 tasks.
    using (SemaphoreSlim semaphore = new SemaphoreSlim(5))
    {
        foreach (var question in questions)
        {
            // Wait until an available slot is free.
            await semaphore.WaitAsync();

            // Create and start a task for each question.
            tasks.Add(Task.Run(async () =>
            {
                try
                {
                    logger.Log($"Processing question: {question}");
                    Stopwatch stopwatch = Stopwatch.StartNew();

                    // Run the process for the question (with validation enabled).
                    string result = await RunProcess(openAIClient, question, logger, validate);

                    stopwatch.Stop();
                    long elapsedMs = stopwatch.ElapsedMilliseconds;
                    logger.Log($"Processed question in {elapsedMs} ms: {question}");
                    return elapsedMs;
                }
                finally
                {
                    // Always release the semaphore.
                    semaphore.Release();
                }
            }));
        }

        // Wait for all tasks to complete.
        long[] processingTimes = await Task.WhenAll(tasks);

        // Compute performance statistics.
        long totalTime = processingTimes.Sum();
        double averageTime = processingTimes.Any() ? processingTimes.Average() : 0;
        long minTime = processingTimes.Any() ? processingTimes.Min() : 0;
        long maxTime = processingTimes.Any() ? processingTimes.Max() : 0;

        // Log overall statistics.
        logger.Log("Performance statistics (concurrent):");
        logger.Log($"Total questions processed: {processingTimes.Length}");
        logger.Log($"Total processing time: {totalTime} ms");
        logger.Log($"Average processing time: {averageTime:F2} ms");
        logger.Log($"Minimum processing time: {minTime} ms");
        logger.Log($"Maximum processing time: {maxTime} ms");
    }

    // Flush out the complete log.
    await logger.FlushLogsAsync();
}

In [ ]:
await TestPerformanceConcurrent(openAIClient,false);

log: Performance statistics:
log: Total questions processed: 20
log: Total processing time: 92817 ms
log: Average processing time: 4640.85 ms
log: Minimum processing time: 1679 ms
log: Maximum processing time: 23241 ms
=== Process Log End ===

## Testing Prompts Section

In [ ]:

// Call your OpenAI method, presumably an async method that returns a string response
var resp = await CallOpenAI(openAIClient, 
    "Question: who won and what was the score of the Super Sports Championship 2025? Answer: Munich Flying Dolphins the score was 24:1",
    systemMessageValidateAnswer, true);

// Print the result
Console.WriteLine(resp);

In [ ]:


var resp = await CallOpenAI(openAIClient, 
    "Question: who won and what was the score of the Super Sports Championship 2025?",
    systemMessageUnderstandQuestion, true);

// Print the result
Console.WriteLine(resp);